Please go to app.py to run the code . These are all trial & error codes , for future references


In [1]:
from dotenv import load_dotenv # pip install python-dotenv
import os

load_dotenv()
parser_key = os.getenv("llama_parser")#It only parse 1000 pages per day for free or you will have to upgrade the plan 🤑💸
'''https://cloud.llamaindex.ai/api-key'''#and there must be 1200 pages per file max
pinecone_key=os.getenv("pinecone_api_key")
huggingface_key=os.getenv("HUGGINGFACE_TOKEN")
if parser_key:
    print(True)
if pinecone_key:
    print(True)
if huggingface_key:
    print(True)


True
True
True


In [2]:
import nest_asyncio
from llama_parse import LlamaParse


# Apply nest_asyncio to handle asynchronous operations
nest_asyncio.apply()

# Initialize LlamaParse with API key from environment variable
api_key = parser_key  # Ensure your API key is set here or in the environment variables
parser = LlamaParse(
    api_key=api_key,
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
)

# Function to handle loading of documents synchronously
def extract_documents_sync(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Function to handle loading of documents asynchronously
async def extract_documents_async(path):
    if os.path.isfile(path):
        return await parser.aload_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return await parser.aload_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Path to the file or directory containing the documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"

# Extract and print the content of the documents synchronously
documents = extract_documents_sync(path)
for doc in documents:
    print(doc.text[:1000])# Print the first 1000 characters of each 




Started parsing the file under job_id 28b9e3df-ec31-4130-abd9-36cbd7358cc7
# Journal of Physics: Conference Series

PAPER &#8226; OPEN ACCESS

Research on Machine Learning and Its Algorithms and Development

To cite this article: Wei Jin 2020 J. Phys.: Conf. Ser. 1544 012003

View the article online for updates and enhancements.

IOP ebooks

Bringing together innovative digital publishing with leading authors from the global scientific community.

Start exploring the collection-download the first chapter of every title for free.

This content was downloaded from IP address 158.46.154.149 on 03/06/2020 at 13:35
# Research on Machine Learning and Its Algorithms and Development

Wei Jin

Northwestern Polytechnical University Ming De College, Xi’an, Shaanxi, China

Abstract: This article analyzes the basic classification of machine learning, including supervised learning, unsupervised learning, and reinforcement learning. It combines analysis on common algorithms in machine learning, such 

In [13]:
#pip install tf-keras
import os
from dotenv import load_dotenv
from llama_parse import LlamaParse
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Load environment variables
load_dotenv()

# Initialize LlamaParse
parser_key = os.getenv("llama_parser")
parser = LlamaParse(api_key=parser_key, result_type="markdown", num_workers=4, verbose=True)

# Function to extract documents
def extract_documents(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Extract documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"
documents = extract_documents(path)

# Initialize Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create and save FAISS index
db = FAISS.from_documents(documents, embeddings)
db.save_local("faiss_index")

# Load the saved index
loaded_db = FAISS.load_local("faiss_index", embeddings)

# Initialize Hugging Face pipeline for language model
model_name = "google/flan-t5-large"  # You can change this to a different model if needed
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)

# Create prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Use three sentences maximum and keep the answer concise.

{context}

Question: {question}
Answer:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Set up retriever with contextual compression
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=loaded_db.as_retriever(search_kwargs={"k": 4})
)

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# Function to answer questions
def answer_question(query):
    result = qa_chain({"query": query})
    return result["result"], result["source_documents"]

# Example usage
query = "How many machine learning algorithms are there?"
answer, source_docs = answer_question(query)
print(f"Answer: {answer}")
print("\nSource Documents:")
for doc in source_docs:
    print(f"- {doc.metadata['source']}: {doc.page_content[:100]}...")

Started parsing the file under job_id 58f333d4-4132-4586-b981-7c527e39bb2e


AttributeError: 'Document' object has no attribute 'page_content'

pip install langchain langchain-huggingface faiss-cpu transformers torch dotenv llama-parse


In [15]:
import os
from dotenv import load_dotenv
from llama_parse import LlamaParse
from langchain.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load environment variables
load_dotenv()

# Initialize LlamaParse
parser_key = os.getenv("llama_parser")
parser = LlamaParse(api_key=parser_key, result_type="markdown", num_workers=4, verbose=True)

# Function to extract documents
def extract_documents(path):
    if os.path.isfile(path):
        return parser.load_data(path)
    elif os.path.isdir(path):
        files = [os.path.join(path, filename) for filename in os.listdir(path)]
        return parser.load_data(files)
    else:
        raise ValueError(f"Path '{path}' is neither a file nor a directory.")

# Extract documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"
documents = extract_documents(path)

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)

# Split documents into smaller chunks
texts = []
metadatas = []
for doc in documents:
    chunks = text_splitter.split_text(doc.text)
    texts.extend(chunks)
    metadatas.extend([doc.metadata] * len(chunks))

# Initialize Hugging Face embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

# Create and save FAISS index
db = FAISS.from_texts(texts, embeddings, metadatas=metadatas)
db.save_local("faiss_index")

# Load the saved index
loaded_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)



Started parsing the file under job_id 07cdd29f-4ddc-4d0b-859e-4e78ee0e9e80


In [17]:
# ... (previous code remains the same)

# Initialize Hugging Face pipeline for language model
model_name = "google/flan-t5-large"  # Upgraded to a larger model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)
llm = HuggingFacePipeline(pipeline=pipe)

# Create a more detailed prompt template
prompt_template = """Use the following pieces of context to answer the question at the end. 
If you don't know the answer, just say that you don't know, don't try to make up an answer. 
Provide a detailed explanation in your answer, using up to five sentences.

Context: {context}

Question: {question}
Detailed Answer:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])

# Set up retriever with contextual compression
compressor = LLMChainExtractor.from_llm(llm)
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=loaded_db.as_retriever(search_kwargs={"k": 6})  # Increased from 4 to 6
)

# Create RetrievalQA chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=compression_retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

# Function to answer questions
def answer_question(query):
    result = qa_chain.invoke({"query": query})
    return result["result"], result["source_documents"]

# Example usage
query = "Can you summerize the topic?"
answer, source_docs = answer_question(query)
print(query,"\n")
print(f"Answer: {answer}")
print("\nSource Documents:")
for doc in source_docs:
    print(f"- : {doc.page_content[:100]}...")

Can you summerize the topic? 

Answer: The theoretical system continues to mature process, but rely on the machine itself to complete the analysis of data information. In practice, the operation method is to let the machine learn the basic concepts and content, and then give the machine enough freedom to complete a series of content learning, including concepts and content similar to the basic principles, such as tree roots Promotion of Personalized Customization Services.

Source Documents:
- : # 1. Introduction...
- : machine learning theory will be continuously strengthened, and the degree of refinement of the conte...
- : process presents a certain regularity, and the learning content is more systematic [1]...
- : # 4. Research on Machine Learning Development # 4.1. Theoretical System Continues to Mature...
- : process, but rely on the machine itself to complete the analysis of data information. In practice, t...
- : Promotion of Personalized Customization Services...


In [8]:
# Extract documents
path = r"C:\Users\Sayan Maity\OneDrive\Desktop\chat_pdf\Research_on_Machine_Learning_and_Its_Algorithms_an.pdf"
documents = extract_documents(path)

# Inspect the first document to understand its structure
first_document = documents[0]
print(first_document.__dict__)



Started parsing the file under job_id 28bd0b85-5696-4ae2-b668-27ba0ac01195
{'id_': '2b05c1a6-8fa3-4273-b168-e2bdabada1a6', 'embedding': None, 'metadata': {}, 'excluded_embed_metadata_keys': [], 'excluded_llm_metadata_keys': [], 'relationships': {}, 'text': '# Journal of Physics: Conference Series\n\nPAPER &#8226; OPEN ACCESS\n\nResearch on Machine Learning and Its Algorithms and Development\n\nTo cite this article: Wei Jin 2020 J. Phys.: Conf. Ser. 1544 012003\n\nView the article online for updates and enhancements.\n\nIOP ebooks\n\nBringing together innovative digital publishing with leading authors from the global scientific community.\n\nStart exploring the collection-download the first chapter of every title for free.\n\nThis content was downloaded from IP address 158.46.154.149 on 03/06/2020 at 13:35', 'mimetype': 'text/plain', 'start_char_idx': None, 'end_char_idx': None, 'text_template': '{metadata_str}\n\n{content}', 'metadata_template': '{key}: {value}', 'metadata_seperator': 

final


In [ ]:
import os# It is used to interacting with our os , we used it to fetch environment variables
import shutil# is used to handle file operations related to the FAISS index directory. Specifically, it ensures that any existing FAISS index directory is deleted before creating a new one.
import gradio as gr# is for the ui
from dotenv import load_dotenv#to load env_variables from .env file
from llama_parse import LlamaParse#(is a document parsing tool) It allows the system to process and analyze document content, which is essential for further operations like indexing, summarization, and question-answering.
from langchain_community.vectorstores import FAISS#(Facebook AI Similarity Search),(for efficient similarity search and clustering of dense vectors),It enables the system to quickly find and retrieve relevant document sections based on query similarity, which is crucial for effective information retrieval.
from langchain_huggingface import HuggingFaceEmbeddings, HuggingFacePipeline#Embeddings capture semantic meaning, allowing the system to compare and search texts based on their content, rather than just keyword matching,Pipeline provides a simple interface to various NLP tasks using pre-trained models.
from langchain.chains import RetrievalQA# It integrates retrieval mechanisms and language models to provide accurate and contextually relevant answers
from langchain.prompts import PromptTemplate# helps the model understand the input structure and respond appropriately.
from langchain.retrievers import ContextualCompressionRetriever#It retrieves and compresses document content, making it suitable for models with input length limitations.
from langchain.retrievers.document_compressors import LLMChainExtractor#It extracts essential parts of the document to help answer specific queries
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline#AutoTokenizer is used to tokenize the input text, breaking it down into tokens that the model can understand.
#AutoModelForSeq2SeqLM is a pre-trained sequence-to-sequence language model used for generating text based on the input tokens.
#it's used to create a text generation pipeline for generating answers or summaries.
from langchain.text_splitter import RecursiveCharacterTextSplitter#This tool splits long texts into smaller, manageable chunks.

# Load environment variables
load_dotenv()


# Initialize LlamaParse
parser_key = os.getenv("llama_parser")  # Fetches the LlamaParse API key from environment variables.
#It only parse 1000 pages per day for free or you will have to upgrade the plan 🤑💸
'''https://cloud.llamaindex.ai/api-key'''#and there must be 1200 pages per file max
parser = LlamaParse(api_key=parser_key, result_type="markdown", num_workers=4, verbose=True)  # Initializes LlamaParse with the API key and specific settings.
# Using markdown makes the output easier to read and understand.
#num_workers=4, the parsing task can be distributed across four workers, allowing for parallel processing.
#When verbose is set to True, the parser provides detailed logging information about the parsing process.



# Global variables
loaded_db = None  # Will hold the loaded FAISS database.
qa_chain = None  # Will hold the RetrievalQA chain.
document_summary = ""  # Will hold the document summary.
tokenizer = None  # Will hold the tokenizer for the language model.
model = None  # Will hold the language model.

def extract_documents(file):
    return parser.load_data(file.name)## Uses LlamaParse to extract text from the uploaded file.

def process_document(file):
    global loaded_db, qa_chain, document_summary, tokenizer, model# Declare global variables to modify them from anywhere.

    documents = extract_documents(file)

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=200,  # Maximum size of each text chunk.
        chunk_overlap=20,  # Overlap between chunks to maintain context.
        length_function=len,  # Function to determine chunk length.
    )

    texts = []  # List to store text chunks.
    metadatas = []  # List to store metadata for each chunk.
    for doc in documents:  # Loop through each extracted document.
        chunks = text_splitter.split_text(doc.text)  # Split document text into chunks.
        texts.extend(chunks)  # Add chunks to the texts list.
        metadatas.extend([doc.metadata] * len(chunks))  # Add metadata for each chunk.

    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")  # Initialize opensource embeddings model.

    if os.path.exists("faiss_index"):  # Check if FAISS index directory exists.
        shutil.rmtree("faiss_index")  # Remove existing FAISS index directory.
    db = FAISS.from_texts(texts, embeddings, metadatas=metadatas)  # Create FAISS index from texts and embeddings.
    db.save_local("faiss_index")  # Save the FAISS index locally.
    loaded_db = FAISS.load_local("faiss_index", embeddings, allow_dangerous_deserialization=True)  # Load the saved FAISS index.It tells the FAISS library to skip some of the safety checks that are typically enforced during deserialization.


    model_name = "google/flan-t5-large"  # Specify the language model to use.
    tokenizer = AutoTokenizer.from_pretrained(model_name)  # Initialize the tokenizer for the language model.
    model = AutoModelForSeq2SeqLM.from_pretrained(model_name)  # Initialize the language model.

    pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=512)  # Create a text generation pipeline.
    llm = HuggingFacePipeline(pipeline=pipe)  # Initialize HuggingFacePipeline with the text generation pipeline.

    prompt_template = """Use the following pieces of context to answer the question at the end. 
    If you don't know the answer, just say that you don't know, don't try to make up an answer. 
    Provide a detailed explanation in your answer, using up to five sentences.

    Context: {context}

    Question: {question}
    Detailed Answer:"""
    PROMPT = PromptTemplate(template=prompt_template, input_variables=["context", "question"])# Initialize the prompt template.

    compressor = LLMChainExtractor.from_llm(llm)  # Initialize the document compressor.
    compression_retriever = ContextualCompressionRetriever(
        base_compressor=compressor,  # Set the document compressor.
        base_retriever=loaded_db.as_retriever(search_kwargs={"k": 4})  #loaded_db.as_retriever(): This method converts the FAISS index (loaded_db) into a retriever object that can be used to perform similarity searches.{"k": 4} indicates that the retriever should return the top 4 most similar documents or text chunks based on the similarity score.
    )

    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,  # Set the language model.
        chain_type="stuff",  #Choosing the Right Chain Type
        #"stuff": Simple and direct, good for straightforward contexts.
        #"map_reduce": Best for complex or large documents requiring summarization.
        #"refine": Suitable for detailed or nuanced answers requiring refinement.
        #"qa": Optimized for direct question-answering tasks.
        retriever=compression_retriever,  #retriever=compression_retriever: Configures the QA chain to use a retriever that not only fetches relevant documents but also compresses them to ensure the input to the language model is concise and effective.
        return_source_documents=True,  # Indicate to return source documents.
        chain_type_kwargs={"prompt": PROMPT}  # Set additional chain type parameters.
    )

    document_summary = summarize_document(loaded_db)  # Generate a summary of the document.

    return "Document processed successfully. You can now ask questions or view the summary."  # Return a success message.

def summarize_document(db, chunk_size=400):
    all_docs = db.similarity_search("", k=db.index.ntotal)  # Retrieve all documents from the database.
    full_text = " ".join(doc.page_content for doc in all_docs)  # Combine all document texts.
    chunks = [full_text[i:i+chunk_size] for i in range(0, len(full_text), chunk_size)]  # Split the full text into chunks.
    return " ".join(chunks)  # Return the combined chunks as the summary.

def answer_question(query):
    if not query.strip():  # Check if the query is empty.
        return "Please enter a question before clicking the Ask button."
    
    if not loaded_db or not qa_chain:  # Check if the database or QA chain is not initialized.
        return "Please process a document first."

    result = qa_chain.invoke({"query": query})  # Get the answer from the QA chain.
    return result["result"].strip().rstrip("[1]").strip()  # Clean up and return the answer.


# Gradio interface
with gr.Blocks(".gradio-container {background: url(file='./img.jpeg')}") as demo:  # Create a Gradio interface with custom CSS.
    gr.Markdown("# Intelligent Document Q&A System")  # Add a title.

    with gr.Row():
        file_input = gr.File(label="Upload Document")  # Add a file upload input.
        process_button = gr.Button("Process Document", elem_classes=["gradio-button"])  # Add a button to process the document.

    status_output = gr.Textbox(label="Status", elem_classes=["gradio-textbox"])  # Add a textbox for status messages.

    with gr.Row():
        question_input = gr.Textbox(label="Ask a question", elem_classes=["gradio-textbox"])  # Add a textbox for inputting questions.
        answer_output = gr.Textbox(label="Answer", elem_classes=["gradio-textbox"])  # Add a textbox for displaying answers.

    ask_button = gr.Button("Ask", elem_classes=["gradio-button"])  # Add a button to ask a question.

    summary_output = gr.Textbox(label="Document Summary", lines=10, elem_classes=["gradio-textbox"])  # Add a textbox for the document summary.
    summary_button = gr.Button("Generate Summary", elem_classes=["gradio-button"])  # Add a button to generate the summary.

    process_button.click(process_document, inputs=[file_input], outputs=[status_output])  # Define the click action for the process button.
    ask_button.click(answer_question, inputs=[question_input], outputs=[answer_output])  # Define the click action for the ask button.
    summary_button.click(lambda: document_summary, outputs=[summary_output])  # Define the click action for the summary button.

# Launch the interface
demo.launch(share=True)  # Launch the Gradio interface with sharing enabled.


